# EcoTrack: Environmental Data Analysis Platform Demo

This notebook demonstrates the key features of the EcoTrack platform:
- Loading and processing satellite imagery
- Detecting deforestation using multiple algorithms
- Analyzing pollution data from sensors
- Creating visualizations and maps
- Forecasting environmental trends

## 1. Setup and Imports

First, let's import the necessary modules and set up our environment.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
import rasterio
from datetime import datetime, timedelta

# Add the src directory to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import EcoTrack modules
from src.data_processing.data_processing_utils import SatelliteImageProcessor, SensorDataProcessor, DataIntegrator
from src.data_processing.deforestation_detection import DeforestationDetector
from src.data_processing.pollution_monitoring import PollutionAnalyzer
from src.visualization.visualization_dashboard import MapVisualizer, DataVisualizer, PlotlyVisualizer

# Set up matplotlib for inline display
%matplotlib inline
plt.style.use('seaborn')

# Create output directory if it doesn't exist
output_dir = '../output/notebook_demo'
os.makedirs(output_dir, exist_ok=True)

## 2. Generate Demo Data

Let's create some demo data to work with. In a real-world scenario, you would load your own satellite imagery and sensor data.

In [ ]:
# Create demo sensor data
def create_demo_sensor_data():
    """Create demo sensor data for air quality monitoring"""
    locations = [
        {'name': 'Forest Edge', 'latitude': 37.7749, 'longitude': -122.4194, 'location_type': 'forest'},
        {'name': 'City Center', 'latitude': 40.7128, 'longitude': -74.0060, 'location_type': 'urban'},
        {'name': 'Industrial Zone', 'latitude': 33.7490, 'longitude': -84.3880, 'location_type': 'industrial'}
    ]
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2022, 1, 1)
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    np.random.seed(42)
    sensor_data = []
    for location in locations:
        base_pm25, base_o3 = (15, 35) if location['location_type'] == 'urban' else (25, 40) if location['location_type'] == 'industrial' else (5, 20)
        for date in date_range:
            day_of_year = date.dayofyear
            season_factor_pm = 1 + 0.5 * np.sin((day_of_year / 365) * 2 * np.pi + np.pi)
            season_factor_o3 = 1 + 0.7 * np.sin((day_of_year / 365) * 2 * np.pi)
            days_elapsed = (date - start_date).days
            trend_factor = 1 - (days_elapsed / (end_date - start_date).days) * 0.2
            daily_fluctuation = np.random.normal(0, 3)
            pm25 = max(0, base_pm25 * season_factor_pm * trend_factor + daily_fluctuation)
            o3 = max(0, base_o3 * season_factor_o3 * trend_factor + np.random.normal(0, 5))
            sensor_data.append({
                'timestamp': date,
                'sensor_id': f"sensor_{locations.index(location) + 1}",
                'location': location['name'],
                'latitude': location['latitude'],
                'longitude': location['longitude'],
                'location_type': location['location_type'],
                'pm25': round(pm25, 2),
                'o3': round(o3, 2),
                'temperature': round(20 + 15 * np.sin((day_of_year / 365) * 2 * np.pi) + np.random.normal(0, 3), 2),
                'humidity': round(50 + 20 * np.sin((day_of_year / 365) * 2 * np.pi + np.pi) + np.random.normal(0, 5), 2)
            })
    return pd.DataFrame(sensor_data)

# Create sensor data
sensor_data = create_demo_sensor_data()
print(f"Demo sensor data shape: {sensor_data.shape}")
sensor_data.head()